In [66]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.evapo import *
import geopandas as gpd
import tarfile
from pyproj import Transformer
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_bounds
from joblib import Parallel, delayed
from rasterio.transform import from_origin
workhorse = True

if workhorse:
    origin = 'Aldhani/eoagritwin/'
else:
    origin = ''

utm_to_epsg = {
    '28': 32628,  # Western Portugal, Azores
    '29': 32629,  # Western Spain, Portugal
    '30': 32630,  # Spain, France, UK
    '31': 32631,  # France, Benelux, Germany, Western Norway
    '32': 32632,  # Germany, Denmark, Switzerland, Italy, Austria
    '33': 32633,  # Central Europe: Poland, Czechia, Hungary, Croatia, Sweden, Norway
    '34': 32634,  # Eastern Europe: Finland, Baltic States, Romania
    '35': 32635,  # Western Russia, Ukraine
    '36': 32636,  # Russia, Black Sea region
}

def get_common_bounds(bounds_dict):
    minx = min(bounds_dict['UL_X'])
    maxy = max(bounds_dict['UL_Y'])
    maxx = max(bounds_dict['LR_X'])
    miny  =min(bounds_dict['LR_Y'])
    return (minx, miny, maxx, maxy)

In [67]:
# rasterize germany
ger = gpd.read_file(f'/data/{origin}misc/gadm41_DEU_shp/gadm41_DEU_0_reproj_32632.shp')
res = 30
# Use union_bounds to get max entent form scences which GErmany will be rasterized to
minx, miny, maxx, maxy = ger.total_bounds
width = math.ceil((maxx - minx) / res)
height = math.ceil((maxy - miny) / res)

# Recalculate bounds to match exact pixel grid
adjusted_maxx = minx + width * res
adjusted_maxy = miny + height * res

# Create transform from the top-left corner (origin)
transform = from_origin(minx, adjusted_maxy, res, res)

aoi_raster = rasterize(
    [(geom, 1) for geom in ger.geometry],
    out_shape=(height, width),
    transform=transform,
    fill=0,
    dtype='uint8',
    all_touched=True
)

with rasterio.open(
    f'/data/{origin}et/Auxiliary/Landsat_GER_mask/GER_Landsat_mask_small.tif',              # Output file path
    'w',                           # Write mode
    driver='GTiff',                # Format
    height=aoi_raster.shape[0],    # Number of rows
    width=aoi_raster.shape[1],     # Number of columns
    count=1,                       # Number of bands (1 = grayscale)
    dtype='uint8',                 # Data type
    crs=ger.crs,                   # Coordinate reference system
    transform=transform            # Affine transform for spatial reference
) as dst:
    dst.write(aoi_raster, 1)

In [71]:
#### load boundaries for states and rasterize them based on GER_Landsat_mask_small.tif
res = 30
gtiff_driver = gdal.GetDriverByName('GTiff')
drvMemR = gdal.GetDriverByName('MEM')

template_ds = gdal.Open(f'/data/{origin}et/Auxiliary/Landsat_GER_mask/GER_Landsat_mask_small.tif')
temp_gt = template_ds.GetGeoTransform()
temp_proj = template_ds.GetProjection()
temp_x_size = template_ds.RasterXSize
temp_y_size = template_ds.RasterYSize

# load shapefiles
ger_border_path = f'/data/{origin}misc/gadm41_DEU_shp/gadm41_DEU_1.shp'
shp_ds = ogr.Open(ger_border_path)
layer = shp_ds.GetLayer()

attr = getAttributesALL(ger_border_path)

In [81]:
for state in attr['NAME_1']:

    sub = drvMemR.Create('', temp_x_size, temp_y_size, 1, gdal.GDT_Int16)
    sub.SetGeoTransform(temp_gt)
    sub.SetProjection(temp_proj)
    band = sub.GetRasterBand(1)
    band.SetNoDataValue(0)
    layer.SetAttributeFilter((f"\"NAME_1\" = '{state}'"))
    gdal.RasterizeLayer(sub, [1], layer, burn_values=[1])
    layer.SetAttributeFilter(None)
    sub_arr = sub.ReadAsArray()
    
    # get only area with values
    row_sums = np.sum(sub_arr, axis = 1)
    first_row = np.where(row_sums > 0)[0][0]
    last_row = np.where(row_sums > 0)[0][-1]
    col_sums = np.sum(sub_arr, axis = 0)
    first_col = np.where(col_sums > 0)[0][0]
    last_col = np.where(col_sums > 0)[0][-1]

    new_arr = sub_arr[first_row:last_row+1, first_col:last_col+1]
    # adapt geotransform
    new_gt = list(temp_gt)
    new_gt[0] = new_gt[0] + new_gt[1] * first_col
    new_gt[3] = new_gt[3] + new_gt[5] * first_row
    new_gt = tuple(new_gt)

    # Create output raster
    target_ds = gtiff_driver.Create(f'/data/{origin}et/Auxiliary/Landsat_GER_mask/states/{state}.tif', new_arr.shape[1], new_arr.shape[0], 1, gdal.GDT_Byte)
    target_ds.SetGeoTransform(new_gt)
    target_ds.SetProjection(temp_proj)
    band = target_ds.GetRasterBand(1)
    band.SetNoDataValue(0)
    band.WriteArray(new_arr)

    del target_ds

    